This was used to test out code for making a mountain recommender. This code is now used in app.py.

In [3]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [40]:
pkl_file = open('../data/mtn_df.pkl', 'rb')
mtn_df = pickle.load(pkl_file)
pkl_file.close() 

In [41]:
mtn_df.head()

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,...,ability_nums,color_nums,resort_bottom,resort_top,greens,blues,blacks,bbs,lifts,price
0,Over the Rainbow,11924.0,10858.0,1066.0,2404.0,319.0,17.6,50.0,69.0,Expert,...,6,4,10800,13010,13,41,20,26,10,0
1,Zoom,11405.0,11005.0,400.0,839.0,195.0,3.7,55.0,66.0,Expert,...,6,4,10800,13010,13,41,20,26,10,0
2,Avalanche Bowl,11840.0,11405.0,435.0,1141.0,255.0,6.7,42.0,76.0,Expert,...,6,4,10800,13010,13,41,20,26,10,0
3,Tiger's Tail,11856.0,11469.0,386.0,1183.0,136.0,3.7,35.0,61.0,Expert,...,6,3,10800,13010,13,41,20,26,10,0
4,Spillway,11849.0,11631.0,218.0,702.0,198.0,3.2,33.0,41.0,Intermediate,...,4,2,10800,13010,13,41,20,26,10,0


In [47]:
features = ['top_elev_(ft)', 
            'bottom_elev_(ft)', 
            'vert_rise_(ft)', 
            'slope_length_(ft)', 
            'avg_width_(ft)', 
            'slope_area_(acres)', 
            'avg_grade_(%)', 
            'max_grade_(%)', 
            'groomed',
            'resort_bottom',
            'resort_top',
            'greens',
            'blues',
            'blacks',
            'bbs',
            'lifts',
            'price']

X = mtn_df[features].values

In [48]:
X

array([[  1.19240000e+04,   1.08580000e+04,   1.06600000e+03, ...,
          2.60000000e+01,   1.00000000e+01,   0.00000000e+00],
       [  1.14050000e+04,   1.10050000e+04,   4.00000000e+02, ...,
          2.60000000e+01,   1.00000000e+01,   0.00000000e+00],
       [  1.18400000e+04,   1.14050000e+04,   4.35000000e+02, ...,
          2.60000000e+01,   1.00000000e+01,   0.00000000e+00],
       ..., 
       [  8.11000000e+03,   8.10000000e+03,   1.00000000e+01, ...,
          1.70000000e+01,   2.50000000e+01,   0.00000000e+00],
       [  8.11300000e+03,   8.10000000e+03,   1.30000000e+01, ...,
          1.70000000e+01,   2.50000000e+01,   0.00000000e+00],
       [  8.14700000e+03,   8.10000000e+03,   4.70000000e+01, ...,
          1.70000000e+01,   2.50000000e+01,   0.00000000e+00]])

In [49]:
def mtn_recommendations(trail_name, resort_name, X, n=5):
    index = mtn_df.index[(mtn_df['trail_name'] == trail_name) & (mtn_df['resort'] == resort_name)][0]
    trail = X[index].reshape(1,-1)
    cs = cosine_similarity(trail, X)[0]
    mtn_df['cosine_sim'] = cs
    s = mtn_df.groupby('resort').mean()['cosine_sim'].sort_values()[::-1]
    orig_row = mtn_df.loc[[index]].rename(lambda x: 'original')
    return list(s.index[:n])

In [50]:
ss = StandardScaler()
X = ss.fit_transform(X)

In [51]:
recs = mtn_recommendations('So Fine','Copper',X,n=5)
recs

['Arapahoe Basin', 'Monarch', 'Taos', 'Copper', 'Loveland']

In [20]:
cs = cosine_similarity(X[300].reshape(1,-1), X)[0]
cs

array([ 0.4896718 ,  0.50842955,  0.5547464 , ..., -0.81362992,
       -0.79814543, -0.79848461])

In [21]:
mtn_df['cosine_sim'] = cs

In [22]:
mtn_df.groupby('resort').mean()['cosine_sim'].sort_values()[::-1]

resort
Arapahoe Basin    0.485526
Monarch           0.374831
Taos              0.353367
Copper            0.342091
Loveland          0.156471
Vail              0.062289
Winter Park       0.042550
Alpine Meadows   -0.285886
Diamond Peak     -0.344899
Crested Butte    -0.407213
Beaver Creek     -0.437231
Eldora           -0.548374
Name: cosine_sim, dtype: float64

In [23]:
_.index[0]

'Arapahoe Basin'

In [24]:
mtn_df.iloc[300]

trail_name            So Fine
top_elev_(ft)           12219
bottom_elev_(ft)        11748
vert_rise_(ft)            470
slope_length_(ft)        1120
avg_width_(ft)            426
slope_area_(acres)         11
avg_grade_(%)              47
max_grade_(%)              85
ability_level          Expert
resort                 Copper
location                   CO
groomed                     0
colors                     bb
ability_nums                6
color_nums                  4
resort_bottom            9712
resort_top              12312
greens                     21
blues                      25
blacks                     36
bbs                        18
lifts                      23
price                       0
cosine_sim                  1
Name: 300, dtype: object

In [25]:
resort_stats_df = mtn_df[['resort', 'resort_bottom','resort_top','greens','blues','blacks','bbs','lifts','price']].drop_duplicates()

In [26]:
resort_stats_df

,resort,resort_bottom,resort_top,greens,blues,blacks,bbs,lifts,price
0,Loveland,10800,13010,13,41,20,26,10,0
86,Arapahoe Basin,10779,13050,10,30,37,23,8,0
182,Copper,9712,12312,21,25,36,18,23,0
303,Eldora,9200,10800,21,51,19,9,11,0
352,Alpine Meadows,6200,9050,25,42,33,0,42,0
426,Vail,8120,11570,18,29,28,25,31,0
617,Monarch,10789,11952,23,28,36,13,6,0
673,Crested Butte,9100,12162,26,57,14,3,15,0
752,Taos,9206,12481,24,25,33,18,16,0
821,Diamond Peak,6700,8539,10,42,48,0,7,0


In [27]:
results_df = pd.DataFrame(columns=['resort', 'resort_bottom','resort_top','greens','blues','blacks','bbs','lifts','price'])
for rec in recs:
    results_df = results_df.append(resort_stats_df[resort_stats_df['resort'] == rec])

In [28]:
results_df

,resort,resort_bottom,resort_top,greens,blues,blacks,bbs,lifts,price
86,Arapahoe Basin,10779,13050,10,30,37,23,8,0
617,Monarch,10789,11952,23,28,36,13,6,0
752,Taos,9206,12481,24,25,33,18,16,0
182,Copper,9712,12312,21,25,36,18,23,0
0,Loveland,10800,13010,13,41,20,26,10,0


In [29]:
def clean_df_for_recs(df):
    df['groomed'][df['groomed'] == 1] = 'Groomed'
    df['groomed'][df['groomed'] == 0] = 'Ungroomed'
    df['color_names'] = df['colors']
    df['color_names'][df['color_names'] == 'green'] = 'Green'
    df['color_names'][df['color_names'] == 'blue'] = 'Blue'
    df['color_names'][df['color_names'] == 'black'] = 'Black'
    df['color_names'][df['color_names'] == 'bb'] = 'Double Black'
    df = df[['trail_name','resort','location','color_names','groomed','top_elev_(ft)','bottom_elev_(ft)','vert_rise_(ft)','slope_length_(ft)','avg_width_(ft)','slope_area_(acres)','avg_grade_(%)','max_grade_(%)']]
    df.columns = ['Trail Name', 'Resort','Location','Difficulty','Groomed','Top Elev (ft)', 'Bottom Elev (ft)', 'Vert Rise (ft)', 'Slope Length (ft)', 'Avg Width (ft)', 'Slope Area (acres)', 'Avg Grade (%)', 'Max Grade (%)']
    return df

In [31]:
clean_df_for_recs(mtn_df)

,Trail Name,Resort,Location,Difficulty,Groomed,Top Elev (ft),Bottom Elev (ft),Vert Rise (ft),Slope Length (ft),Avg Width (ft),Slope Area (acres),Avg Grade (%),Max Grade (%)
0,Over the Rainbow,Loveland,CO,Double Black,Ungroomed,11924.0,10858.0,1066.0,2404.0,319.0,17.6,50.0,69.0
1,Zoom,Loveland,CO,Double Black,Ungroomed,11405.0,11005.0,400.0,839.0,195.0,3.7,55.0,66.0
2,Avalanche Bowl,Loveland,CO,Double Black,Ungroomed,11840.0,11405.0,435.0,1141.0,255.0,6.7,42.0,76.0
3,Tiger's Tail,Loveland,CO,Black,Ungroomed,11856.0,11469.0,386.0,1183.0,136.0,3.7,35.0,61.0
4,Spillway,Loveland,CO,Blue,Groomed,11849.0,11631.0,218.0,702.0,198.0,3.2,33.0,41.0
5,Waterfall,Loveland,CO,Blue,Ungroomed,11638.0,11515.0,124.0,370.0,172.0,1.5,36.0,45.0
6,Upper Richard's,Loveland,CO,Blue,Ungroomed,11816.0,11702.0,113.0,348.0,159.0,1.3,35.0,38.0
7,Cat Walk,Loveland,CO,Green,Groomed,11856.0,11644.0,212.0,1830.0,63.0,2.7,12.0,31.0
8,Holy Cat,Loveland,CO,Black,Ungroomed,11141.0,10917.0,224.0,505.0,90.0,1.0,50.0,67.0
9,Busy Gully,Loveland,CO,Black,Ungroomed,11421.0,10903.0,518.0,1394.0,115.0,3.7,41.0,65.0


In [32]:
results_df

,resort,resort_bottom,resort_top,greens,blues,blacks,bbs,lifts,price
86,Arapahoe Basin,10779,13050,10,30,37,23,8,0
617,Monarch,10789,11952,23,28,36,13,6,0
752,Taos,9206,12481,24,25,33,18,16,0
182,Copper,9712,12312,21,25,36,18,23,0
0,Loveland,10800,13010,13,41,20,26,10,0


In [33]:
results_df.colummns = ['Resort','Resort Bottom Elevation', 'Resort Top Elevations', 'Percent Greens', 'Percent Blues', 'Percent Blacks', 'Percent Double  Blacks', 'Number of Lifts', 'Price']

In [34]:
for i,row in results_df.iterrows():
    print(row.resort)

Arapahoe Basin
Monarch
Taos
Copper
Loveland


In [46]:
recs = mtn_recommendations('So Fine','Copper',X,n=5)
recs

['Arapahoe Basin', 'Monarch', 'Loveland', 'Taos', 'Copper']